In [6]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def get_cursor(connection):
    try:
        cursor = connection.cursor()
        print("cursor creation successful")
    except Error as e:
        print(f"the error {e} occurred")
    
    return cursor

def create_table(cursor, script_path):
    try:
        sql_file = open(script_path)
        sql_as_string = sql_file.read()
        cursor.executescript(sql_as_string)
        print("table successfully created")
    except Error as e:
        print(f"the error {e} occurred")

        
def df_to_sql(df, connection, table, if_exists):
    try:
        df.to_sql(name=table, con=connection, if_exists=if_exists, index = False)
    except Error as e:
        print(f"the error {e} occurred")


def delete_table(cursor, table):
    try:
        cursor.execute(f"DROP TABLE {table}")
        print("Table dropped... ")
    except:
        print(f"the error {e} occurred")

def close_connection(connection):
    try:
        connection.commit()
        connection.close()
        print("successfully closed")
    except Error as e:
        print(f"the error {e} occurred")

def load_content(cursor, table, limit):
    try:
        cursor.execute(f'''  
        SELECT * FROM {table} LIMIT {limit}
                ''')
        for row in cursor.fetchall():
            print (row)

    except Error as e:
        print(f"the error {e} occurred")


In [18]:
connection = create_connection("C:\\10\week11\ELT_Pipeline_Project\warehouse\warehouse.db")

Connection to SQLite DB successful


In [19]:
cursor = get_cursor(connection)

cursor creation successful


In [19]:
import pandas as pd

data = {'product_name': ['Computer','Tablet','Monitor','Printer'],
        'price': [900,300,450,150]
        }

df = pd.DataFrame(data, columns= ['product_name','price'])
print (df)

  product_name  price
0     Computer    900
1       Tablet    300
2      Monitor    450
3      Printer    150


In [22]:
create_table(cursor, "../scripts/sql/sample.sql")

table successfully created


In [28]:
df_to_sql(df, connection, "products", "append")

In [31]:
load_content(cursor, "products", 2)

('Computer', 900)
('Tablet', 300)


In [7]:
data = pd.read_csv("../data/sample.csv")

In [8]:
data.head()

,track_id; type; traveled_d; avg_speed; lat; lon; speed; lon_acc; lat_acc; time
0,1; Car; 48.85; 9.770344; 37.977391; 23.737688;...
1,2; Motorcycle; 98.09; 19.839417; 37.977642; 23...
2,3; Motorcycle; 63.80; 18.228752; 37.977997; 23...
3,4; Motorcycle; 145.72; 26.229014; 37.978135; 2...
4,5; Motorcycle; 138.01; 24.841425; 37.978134; 2...


In [9]:
columns = data.columns[0].split(";")[:4]
columns.append("trajectory")
columns[1] = "types"
for i in range(len(columns)):
    columns[i] = columns[i].strip()
columns

['track_id', 'types', 'traveled_d', 'avg_speed', 'trajectory']

In [10]:

track_ids = []
types = []
traveled_d = []
avg_speeds = []
trajectories = []

for r in range(len(data)): 
    row = data.iloc[r,:][0].split(";")
    row_p1 = row[:4]
    row_p2 = row[4:]
    trajectory = ','.join(row_p2)
    
    track_ids.append(row_p1[0])
    types.append(row_p1[1])
    traveled_d.append(row_p1[2])
    avg_speeds.append(row_p1[3])
    trajectories.append(trajectory[1:])


In [11]:
data_dict = {columns[0]:track_ids, columns[1]:types, columns[2]:traveled_d, columns[3]:avg_speeds, columns[4]:trajectories}

In [12]:
new_df = pd.DataFrame(data_dict)
new_df.head()

,track_id,types,traveled_d,avg_speed,trajectory
0,1,Car,48.85,9.770344,"37.977391, 23.737688, 4.9178, 0.0518, -0.0299,..."
1,2,Motorcycle,98.09,19.839417,"37.977642, 23.737400, 16.9759, -0.0361, -0.022..."
2,3,Motorcycle,63.80,18.228752,"37.977997, 23.737264, 20.1906, -0.0795, -0.339..."
3,4,Motorcycle,145.72,26.229014,"37.978135, 23.737072, 2.7555, -0.0302, 0.0948,..."
4,5,Motorcycle,138.01,24.841425,"37.978134, 23.737103, 0.0000, 0.0000, 0.0000, ..."


In [20]:
create_table(cursor, "../scripts/sql/source_table.sql")

table successfully created


In [101]:
delete_table(cursor, "source")

Table dropped... 


In [21]:
df_to_sql(new_df, connection, "source", "append")

In [22]:
close_connection(connection)

successfully closed
